## 1. Genetický algoritmus

Tento kód implementuje genetický algoritmus (GA) na optimalizáciu stratégií pre hru Blackjack. Cieľom je nájsť optimálnu stratégiu rozhodovania sa (**hit/stand**), ktorá maximalizuje očakávaný výnos hráča.

### Parametre genetického algoritmu
- **population_size** (predvolená hodnota: 50): Veľkosť populácie (počet stratégií v každej generácii)
- **mutation_rate** (predvolená hodnota: 0.01): Pravdepodobnosť mutácie každého rozhodnutia v stratégii
- **generations** (predvolená hodnota: 100): Počet generácií pre GA
- **episodes** (nastavené na 2): Počet epizód vyhodnocovania stratégie

### Inicializácia matíc

Algoritmus inicializuje populáciu stratégií pre Blackjack, kde každá stratégia pozostáva z dvoch matíc:

- **Matica hard_totals (17x10):**
    - Riadky predstavujú súčet hráčových kariet (4-20)
    - Stĺpce predstavujú kartu dealera (2-10, A)
    - Hodnoty 0/1 označujú akciu (0=stand, 1=hit)

- **Matica soft_totals (9x10):**
    - Riadky predstavujú súčet hráčových kariet s použiteľným esom (12-20)
    - Stĺpce predstavujú kartu dealera (2-10, A)
    - Hodnoty 0/1 označujú akciu (0=stand, 1=hit)

Inicializácia prebieha v metóde `create_blackjack_matrices()`, kde sa pre každú pozíciu v maticiach náhodne vyberie hodnota 0 alebo 1 (stand alebo hit).

### Evolučný proces

Genetický algoritmus iteratívne vylepšuje stratégie v priebehu generácií:

1. **Vyhodnotenie fitness:**
     - Každá stratégia je vyhodnotená simuláciou hier Blackjacku.
     - Fitness je priemerná výhra/prehra naprieč týmito hrami.

2. **Selekcia:**
     - Vyberie sa najlepšia polovica stratégií podľa ich fitness hodnoty.
     - Tieto stratégie sa stanú rodičmi pre novú generáciu.

3. **Elitizmus:**
     - Najlepšie 2 stratégie z aktuálnej generácie sú zachované bez zmeny.
     - To zabezpečuje, že kvalita populácie neklesne.

4. **Kríženie:**
     - Nová stratégia vzniká kombináciou dvoch rodičovských stratégií.
     - Pre každú pozíciu v matici je 50% šanca, že hodnota bude zdedená od prvého rodiča, inak od druhého.
     - Kríženie sa vykonáva samostatne pre `hard_totals` aj `soft_totals`.

5. **Mutácia:**
     - S malou pravdepodobnosťou (`mutation_rate`, defaultne 0.01) sa hodnoty v maticiach náhodne zmenia.
     - Mutácia prebehne inverziou hodnoty (0→1 alebo 1→0).
     - Umožňuje algoritmu skúmať nové možnosti mimo aktuálnej populácie.

Algoritmus sleduje najlepšiu stratégiu nájdenú počas celého evolučného procesu a jej fitness hodnotu. Po dokončení všetkých generácií vráti túto najlepšiu stratégiu spolu s históriou fitness pre analýzu konvergencie algoritmu.

### Cieľ experimentu
Zistiť, za akých podmienok sa genetický algoritmus dokáže najviac priblížiť k optimálnej stratégii. Experiment systematicky testuje rôzne kombinácie parametrov a porovnáva výsledné stratégie s známou optimálnou stratégiou.


### Štruktúra experimentu

Experiment testuje kombinácie nasledujúcich parametrov:

- **Veľkosť populácie:** 10, 20  
- **Miera mutácie:** 0,01, 0,05  
- **Počet generácií:** 50, 100  
- **Počet epizód:** 10 000, 20 000  

---

### Metodológia vyhodnotenia

### Stanovenie základnej úrovne

- Výkon **optimálnej stratégie** je vyhodnotený pre 10 000 aj 20 000 epizód  
- Tieto slúžia ako **referenčné body** pre hodnotenie geneticky vyvinutých stratégií  

### Evolúcia stratégie

- Pre každú kombináciu parametrov je spustený **genetický algoritmus**  
- Trieda `BlackjackGA` riadi evolučný proces s určenými parametrami  
- **Najlepšia stratégia** z každého behu je zaznamenaná  

### Hodnotenie výkonu

- Každá vyvinutá stratégia je vyhodnotená **v porovnaní s optimálnou stratégiou**  
- Metrika `reward_difference` kvantifikuje, ako veľmi sa každá stratégia **odchyľuje od optimálnej**


In [ ]:
from genetic_algorithm import BlackjackGA
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import numpy as np
from evaluate_utils import evaluate_strategy, optimal_hard_totals, optimal_soft_totals

# Konfigurácie experimentu
population_sizes = [10, 20]
mutation_rates = [0.01, 0.05]
generations_list = [50, 100]
episode_counts = [10000, 20000]

optimal_reward_10000 = evaluate_strategy((optimal_hard_totals, optimal_soft_totals), total_episodes=10000)
optimal_reward_20000 = evaluate_strategy((optimal_hard_totals, optimal_soft_totals), total_episodes=20000)

# Príprava dataframe pre výsledky
results = []

# Experimentovanie s rôznymi parametrami
for pop_size in tqdm(population_sizes, desc="Population Sizes"):
    for mutation in mutation_rates:
        for gens in generations_list:
            for episode_count in episode_counts:
                # Spustenie algoritmu s aktuálnymi parametrami
                solver = BlackjackGA(population_size=pop_size, mutation_rate=mutation, generations=gens, episodes=episode_count)
                if episode_count == 10000:
                    optimal_reward = optimal_reward_10000
                else:
                    optimal_reward = optimal_reward_20000

                best_strategy, best_fitness, fitness_history = solver.run()
                
                # Vyhodnotenie výsledkov
                output_reward = evaluate_strategy(best_strategy, total_episodes=episode_count)
                
                # Výpočet rozdielu medzi výstupnou odmenou a optimálnou odmenou
                reward_difference = optimal_reward - output_reward

                # Uloženie výsledkov
                results.append({
                    'population_size': pop_size,
                    'mutation_rate': mutation,
                    'generations': gens,
                    'episodes': episode_count,
                    'output_reward': output_reward,
                    'optimal_reward': optimal_reward,
                    'reward_difference': reward_difference
                })

            

# Konverzia na dataframe
results_df = pd.DataFrame(results)

# Zobrazenie výsledkov
print(results_df.head())

# Vizualizácia vplyvu parametrov na zhodu s optimálnou stratégiou
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
sns.boxplot(x='population_size', y='reward_difference', data=results_df)
plt.title('Vplyv veľkosti populácie na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 2)
sns.boxplot(x='mutation_rate', y='reward_difference', data=results_df)
plt.title('Vplyv miery mutácie na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 3)
sns.boxplot(x='generations', y='reward_difference', data=results_df)
plt.title('Vplyv počtu generácií na zhodu s optimálnou stratégiou')

plt.subplot(2, 2, 4)
sns.boxplot(x='episodes', y='reward_difference', data=results_df)
plt.title('Vplyv počtu epizod na zhodu s optimálnou stratégiou')

plt.tight_layout()
plt.show()

# Vytvorenie tepelnej mapy rozdielov medzi najlepšou nájdenou stratégiou a optimálnou
best_result_idx = results_df['reward_difference'].idxmax()
best_params = results_df.iloc[best_result_idx]

print(f"Best parameters: population_size={best_params['population_size']}, " +
      f"mutation_rate={best_params['mutation_rate']}, generations={best_params['generations']}, " +
      f"elite_count={best_params['elite_count']}")

# Spustíme algoritmus s najlepšími parametrami pre vizualizáciu
best_solver = BlackjackGA(
    population_size=int(best_params['population_size']), 
    mutation_rate=best_params['mutation_rate'], 
    generations=int(best_params['generations']),
    episodes=int(best_params['episode_count'])
)
best_strategy, fitness, fitness_history = best_solver.run()

# Vizualizácia histórie fitness
plt.figure(figsize=(10, 6))
plt.plot(fitness_history, label='Fitness History', color='blue')
plt.axhline(y=best_fitness, color='red', linestyle='--', label='Best Fitness')
plt.title('Fitness History Across Generations')
plt.xlabel('Generation')
plt.ylabel('Fitness')
plt.legend()
plt.grid()
plt.show()

# 2. Neurónová sieť

Tento kód implementuje model neurónovej siete, ktorá sa učí optimálnu stratégiu pre hru Blackjack. Model je trénovaný pomocou supervised learning na vopred definovanej optimálnej stratégii a následne je schopný predpovedať akcie (hit/stand) pre rôzne herné stavy.

### Architektúra siete

```python
layer_configs = [
    [(32, 'relu'), (16, 'relu')],
    [(64, 'relu'), (32, 'relu')],
    [(128, 'relu'), (64, 'relu')]
]
```

Architektúra pozostáva z:

- **Vstupnej vrstvy** pre 3 parametre:
  - skóre hráča
  - karta dealera
  - použiteľné eso
- **Skrytých vrstiev**:
  - **[(32, 'relu'), (16, 'relu')]:** Dve skryté vrstvy s 32 a 16 neurónmi, obe s aktivačnou funkciou ReLU. 
  - **[(64, 'relu'), (32, 'relu')]:** Dve skryté vrstvy s 64 a 32 neurónmi.
  - **[(128, 'relu'), (64, 'relu')]:** Dve skryté vrstvy s 128 a 64 neurónmi.
- **Výstupnej vrstvy** s aktivačnou funkciou **sigmoid**, ktorá určuje pravdepodobnosť akcie:
  - `1 = hit`, `0 = stand`

---

### Trénovacie dáta

Trénovacie dáta sú generované funkciou `create_training_data()`, ktorá vytvára páry **(stav, akcia)** podľa základnej stratégie Blackjacku.

### Kombinácie:

- **Skóre hráča**: 4–21  
- **Karta dealera**: 1–10 (kde `1 = Eso`, `10 = 10/J/Q/K`)  
- **Stav esa**: použiteľné / nepoužiteľné

### Normalizácia vstupov (na rozsah [0,1]):

- Skóre hráča: `(player_sum - 4) / 17.0`  
- Karta dealera: `(dealer_card - 1) / 9.0`  
- Použiteľné eso: `1` alebo `0`

### Určenie akcií:

- Podľa základnej stratégie pre:
  - **Tvrdé súčty** (bez použiteľného esa)
  - **Mäkké súčty** (s použiteľným esom)

---

### Parametre trénovania

Trénovanie prebieha v metóde `train()` s nasledujúcimi parametrami:

- `epochs` *(predvolene 100)*: počet trénovacích epoch  
- `batch_size` *(predvolene 32)*: veľkosť dávky pre trénovanie  
- **Validácia** na 10 % trénovacích dát  
- **Optimalizátor**: `Adam`  
- **Loss funkcia**: `binary_crossentropy`  
- **Metrika**: `accuracy`

---

### Vyhodnocovanie modelu

Model je vyhodnocovaný metódou `evaluate()`, ktorá simuluje hru Blackjack pomocou knižnice **Gymnasium**:

- Prebehne `num_episodes` hier *(predvolene 2000)*
- V každej hre:
  - Model určuje akciu (`hit` / `stand`) podľa aktuálneho stavu
  - Zaznamenáva sa priemerna odmena


### Priebeh experimentu

1. **Výpočet priemernej odmeny** známej optimálnej stratégie.
2. **Pre každú konfiguráciu siete**:
  - Vytvorenie a trénovanie modelu neurónovej siete.
  - Vyhodnotenie jej výkonu v porovnaní s optimálnou stratégiou.
  - Zaznamenanie metrík pre porovnanie.
3. **Vizualizácia a analýza výsledkov**.

In [ ]:
from neural_network import BlackjackNN  # Only import BlackjackNN from neural_network
from evaluate_utils import optimal_hard_totals, optimal_soft_totals, evaluate_strategy # Keep other imports as they are
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
    
layer_configs = [
    [(32, 'relu'), (16, 'relu')],
    [(64, 'relu'), (32, 'relu')],
    [(128, 'relu'), (64, 'relu')]
]

optimal_reward = evaluate_strategy((optimal_hard_totals, optimal_soft_totals))

results = []

for config in layer_configs:
    print(f"\nTraining with layer configuration: {config}")
    learner = BlackjackNN(layer_config=config)
    history = learner.train(epochs=100, batch_size=32)
    
    model_reward = learner.evaluate()
    
    #Save results
    config_name = '-'.join([f"{units}" for units, _ in config])
    results.append({
        'config': config_name,
        'optimal_reward': optimal_reward,
        'model_reward': model_reward,
        'reward_difference': abs(optimal_reward) - abs(model_reward),
        'accuracy': max(history.history['val_accuracy']),
        'num_layers': len(config),
        'total_neurons': sum([units for units, _ in config])
    })
    
    print(f"Optimal Strategy Avg Reward: {optimal_reward:.4f}")
    print(f"Model Strategy Avg Reward: {model_reward:.4f}")
    print(f"Difference: {optimal_reward - model_reward:.4f}")
    print(f"Validation Accuracy: {max(history.history['val_accuracy']):.4f}")

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Create visualizations
plt.figure(figsize=(15, 10))

# Plot reward comparison
plt.subplot(2, 2, 1)
sns.barplot(x='config', y='model_reward', data=results_df, color='blue', label='Model')
sns.barplot(x='config', y='optimal_reward', data=results_df, color='green', alpha=0.5, label='Optimal')
plt.title('Model vs Optimal Strategy Reward')
plt.xlabel('Network Configuration (neurons per layer)')
plt.ylabel('Average Reward')
plt.legend()

# Plot reward difference
plt.subplot(2, 2, 2)
sns.barplot(x='config', y='reward_difference', data=results_df)
plt.title('Reward Difference (Optimal - Model)')
plt.xlabel('Network Configuration (neurons per layer)')
plt.ylabel('Difference')

# Plot accuracy vs. network size
plt.subplot(2, 2, 3)
sns.scatterplot(x='total_neurons', y='accuracy', size='num_layers', data=results_df, sizes=(100, 200))
plt.title('Accuracy vs Network Size')
plt.xlabel('Total Neurons')
plt.ylabel('Validation Accuracy')

# Plot accuracy vs. reward difference
plt.subplot(2, 2, 4)
sns.scatterplot(x='accuracy', y='reward_difference', hue='config', data=results_df, s=100)
plt.title('Accuracy vs Reward Difference')
plt.xlabel('Validation Accuracy')
plt.ylabel('Reward Difference (Optimal - Model)')

plt.tight_layout()
plt.savefig('network_configurations_comparison.png')
plt.show()

#Print summary
print("\nSummary of Network Configurations:")
summary = results_df[['config', 'model_reward', 'optimal_reward', 'reward_difference', 'accuracy']]
summary = summary.sort_values('reward_difference')
print(summary.to_string(index=False, float_format=lambda x: f"{x:.4f}"))


# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss During Training')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
    
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy During Training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
    
plt.tight_layout()
plt.savefig('training_history.png')
plt.show()